In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import xml.etree.ElementTree as ET
from typing import *
import time
from utils import *
import threading
import queue
import pandas as pd

NCBI_API_KEY = os.getenv("NCBI_API_KEY")
NCBI_EMAIL = os.getenv("email")

In [3]:
def make_esearch_call(query: str, api_key: None | str, email: str = "your_email@example.com") -> Tuple[int, str, str]:
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

    params = {
        "db": "pubmed",
        "term": query,
        "usehistory": "y",
        "retmode": "json",
        "retmax": 0,
        "email": email
    }

    if api_key:
        params["api_key"] = api_key

    res = requests.get(url, params=params)
    res.raise_for_status()

    data = res.json()["esearchresult"]

    count = int(data["count"])
    webenv = data["webenv"]
    query_key = data["querykey"]

    print(f"Total results: {count}")
    
    return count, webenv, query_key

In [ ]:
query = "covid-19 AND 2024[dp] AND humans[MeSH Terms]"

count, webenv, query_key = make_esearch_call(query, NCBI_EMAIL,NCBI_API_KEY)

Total results: 40507


In [32]:
# RETMAX=1000

# params = {
#             "db": "pubmed",
#             "query_key": query_key,
#             "WebEnv": webenv,
#             "retstart": 0,
#             "retmax": RETMAX,
#             "retmode": "xml",
#             "email": NCBI_EMAIL,
#             "api_key": None
#         }
# res=None
# try:
#     res = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi", params=params)
#     res.raise_for_status()

#     filename = f"pubmed_batch_{0}_{0 + RETMAX - 1}.xml"

#     with open(filename, "w", encoding="utf-8") as f:
#         f.write(res.text)

#     print(f"[EF-{1}] Fetched records {0}-{0 + RETMAX - 1}")

# except Exception as e:
#     print(f"[EF-{1}] Error at start={0}: {e}")
# finally:
#     q.task_done()
#     time.sleep(1.0 / RATE_LIMIT)  # To respect NCBI limit